In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from torchvision.datasets import ImageFolder
from sklearn.model_selection import train_test_split

In [9]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Directory setup
base_dir = '/content/drive/MyDrive/JPEGImages'

# Data transforms
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((160, 160)),  # Slightly smaller than typical ResNet input
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((160, 160)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

# Loading datasets with PyTorch ImageFolder
train_data = ImageFolder(root=base_dir, transform=data_transforms['train'])
train_idx, val_idx = train_test_split(list(range(len(train_data))), test_size=0.2, random_state=42, stratify=train_data.targets)

train_sampler = torch.utils.data.SubsetRandomSampler(train_idx)
val_sampler = torch.utils.data.SubsetRandomSampler(val_idx)

# Data loaders
train_loader = DataLoader(train_data, batch_size=64, sampler=train_sampler, num_workers=4, pin_memory=True)
val_loader = DataLoader(train_data, batch_size=64, sampler=val_sampler, num_workers=4, pin_memory=True)


In [10]:
# Model setup
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_data.classes))  # Replace the fully connected layer

# Transfer model to GPU
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 189MB/s]


In [11]:
# Train the model
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    start_time = time.time()

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader, 0):
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward and optimize
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if i % 20 == 19:  # Print every 20 mini-batches
                print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 20:.4f}')
                running_loss = 0.0

        scheduler.step()

    total_time = time.time() - start_time
    print(f'Training complete in {total_time // 60:.0f}m {total_time % 60:.0f}s')


In [12]:
# Evaluate the model
def evaluate_model(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the validation images: {100 * correct / total:.2f}%')


In [13]:

# Run training and evaluation
train_model(model, criterion, optimizer, scheduler, num_epochs=10)
evaluate_model(model, val_loader)


Epoch 1, Batch 20, Loss: 2.4912
Epoch 1, Batch 40, Loss: 1.7370
Epoch 1, Batch 60, Loss: 1.5117
Epoch 1, Batch 80, Loss: 1.4061
Epoch 1, Batch 100, Loss: 1.3978
Epoch 1, Batch 120, Loss: 1.3424
Epoch 2, Batch 20, Loss: 1.0268
Epoch 2, Batch 40, Loss: 1.0271
Epoch 2, Batch 60, Loss: 0.9908
Epoch 2, Batch 80, Loss: 0.9331
Epoch 2, Batch 100, Loss: 0.9648
Epoch 2, Batch 120, Loss: 1.0481
Epoch 3, Batch 20, Loss: 0.7871
Epoch 3, Batch 40, Loss: 0.7595
Epoch 3, Batch 60, Loss: 0.7890
Epoch 3, Batch 80, Loss: 0.7644
Epoch 3, Batch 100, Loss: 0.8337
Epoch 3, Batch 120, Loss: 0.7163
Epoch 4, Batch 20, Loss: 0.5664
Epoch 4, Batch 40, Loss: 0.5446
Epoch 4, Batch 60, Loss: 0.6532
Epoch 4, Batch 80, Loss: 0.6297
Epoch 4, Batch 100, Loss: 0.6141
Epoch 4, Batch 120, Loss: 0.5621
Epoch 5, Batch 20, Loss: 0.5115
Epoch 5, Batch 40, Loss: 0.4970
Epoch 5, Batch 60, Loss: 0.4785
Epoch 5, Batch 80, Loss: 0.4648
Epoch 5, Batch 100, Loss: 0.5024
Epoch 5, Batch 120, Loss: 0.4839
Epoch 6, Batch 20, Loss: 0.386